## Make conjoint features
## Christian Roncal CMSC499A Dr. Leiserson 

### Todo: Make this a python script

In [1]:
import numpy as np
import pickle
from Bio import SeqIO
from Bio.Alphabet import ProteinAlphabet
import json

In [2]:
D = pickle.load(open('D.pkl', 'rb'))

In [3]:
classes = dict.fromkeys(['A', 'G','V'], 1)
classes.update(dict.fromkeys(['I', 'L', 'F', 'P'], 2))
classes.update(dict.fromkeys(['Y', 'M', 'T', 'S'], 3))
classes.update(dict.fromkeys(['H', 'N', 'Q', 'W'], 4))
classes.update(dict.fromkeys(['R', 'K'], 5))
classes.update(dict.fromkeys(['D', 'E'], 6))
classes.update(dict.fromkeys(['C', 'U'], 7))

In [4]:
def protToClass(p):
    if p == 'X':
        return '-1'
    else:
        return str(classes[p])

def seqToClass(seq):
    return ''.join(list(map(protToClass, seq)))

def normalize(Fi):
    return (Fi - min(Fi)) / max(Fi)

def getFi(D, seq):
    grptoi = {p:i for i,p in enumerate(D)} # group to index mappings
    Fi = np.zeros(len(grptoi.values()))
    
    classSeq = seqToClass(seq)
#     print(classSeq)
    
    for p in D:
        Fi[grptoi[p]] += classSeq.count(''.join(p))

    return normalize(Fi)

In [5]:
def proteinize(records):
    for r in records:
        r.seq.Alphabet = ProteinAlphabet

In [6]:
def makeF(records):
    n = len(records)
    
    F = np.zeros((n, 2793 + 1))
    uniprottoi = {}
    itouniprot = {}
    
    
    for i in range(n):
        F[i,:] = np.append(getFi(D, records[i]), [1]) # add bias
        virus_name = records[i].name.split('|')[1]
        uniprottoi[virus_name] = i
        itouniprot[i] = virus_name
        
    return F, uniprottoi, itouniprot

In [7]:
virus_records = list(SeqIO.parse('virus_prots.fasta', 'fasta'))
human_records = list(SeqIO.parse('human_prots.fasta', 'fasta'))

In [8]:
proteinize(virus_records)

In [9]:
print(virus_records[0].name.split('|')[1]) # to get names of viruses

Q9WMX2


In [10]:
vfeats, vtoi, itov = makeF(virus_records)

In [11]:
'B4UPB0' in vtoi.keys()

False

In [11]:
np.savetxt('vfeats.txt', vfeats)

In [12]:
with open('vtoi.json', 'w') as f:
    json.dump(vtoi, f)
    
with open('itov.json', 'w') as f:
    json.dump(itov, f)

In [13]:
v = np.loadtxt('vfeats.txt')

In [14]:
hfeats, htoi, itoh = makeF(human_records)

(3302, 2794)


In [15]:
with open('htoi.json', 'w') as f:
    json.dump(htoi, f)
    
with open('itoh.json', 'w') as f:
    json.dump(itoh, f)

In [16]:
np.savetxt('hfeats.txt', hfeats)

In [17]:
h = np.loadtxt('hfeats.txt')

In [18]:
h.shape

(3302, 2794)